# **Logbook es. 10**

## **Introduzione**

Con la seguente esperienza si volgiono esplorare capacità e funzionamento del sensore inerziale fornitoci per la pauda invernale.

### **Task 1**

Obiettivo: Scegliere due canali digitali a piacere fra i 16 disponibili con nomi che vanno da DIO0 (digital input/output numero 0) a DIO15 (digital input/output numero 15), in seguito i canali scelti verranno indicati con A e B. Dopodichè eseguire le sequenti azioni:

* Selezionare la modalità Patterns (generatore di segnali digitali) e aggiungere un singolo segnale su A, selezionare output DD, tipo Random, una frequenza di 500Hz e avviare la generazione con il tasto RUN;

* Cortocircuitare A con uno degli ingressi analogici di Analog Discovery 2, per esempio Ch1 (ricordare la terra!), e verificare la corretta generazione di un segnale di tipo digitale;

* Selezionare la modalità Logic (oscilloscopio digitale) e definire un bus a cui aggiungere il secondo canale digitale scelto B: scollegare A dall’ingresso analogico e collegarlo a B per verificare la corretta misura
ed identificazione dei livelli digitali.

Si sono scelte le porte digitali DIO1 e DIO2, rispettivamente A e B.

Prima si è impostato il canale A, ottenendo il seguente risultato:

<img src="DIO1_task1.png" alt="impostazione DIO1" width="800" height="50"/>

*Figura 1: risultato ottenuto impostando DIO1 come richiesto.*

Poi si osservata l'uscita della porta all'oscilloscopio:

<img src="osc_DIO1_task1.png" alt="oscilloscopio DIO1" width="800" />

*Figura 2: osservazione all'oscilloscopio di DIO1.*

Dopo di che si è anche impostato il canale DIO2 come bus, impostando anche il clock per questo canale: DIO1.

<img src="bus_task1.png" alt="bus DIO2" width="1000"  height="100"/>

*Figura 3: osservazione all'oscilloscopio di DIO1.*

### Task 2 – Parametri dal datasheet del MPU6050

**Obiettivo:**
- Estrarre dal datasheet, per le grandezze misurate (accelerazione, velocità angolare, temperatura):
  - **range dinamico**,
  - **sensibilità** (LSB per unità fisica),
  - **accuratezza** (tolleranze di calibrazione, offset, ecc.),
  - **precisione** (rumore, ripetibilità, dipendenze da configurazione e temperatura).

**Descrizione:**
- Si consultano le tabelle delle specifiche elettriche (sezioni 6.1–6.3 del datasheet) per accelerometro, giroscopio e sensore di temperatura.
- Per ogni grandezza si compilano tabelle riassuntive (range, scale factor, tolleranze, valori tipici di rumore, ecc.).
- Si discute il significato fisico di:
  - *range dinamico*,
  - *sensibilità* (LSB/g o LSB/(°/s)),
  - *accuratezza* (es. Initial Calibration Tolerance, zero-g output, zero-rate output),
  - *precisione* (rumore RMS, distribuzione delle misure).
- Si mettono in relazione questi parametri con l’uso pratico del sensore: scelta del fondoscala, intervallo di validità delle misure, limiti di risoluzione.

**GIROSCOPIO** (specifiche di voltaggio):
| Parametro                     | Valore                         |
|-------------------------------|--------------------------------|
| $V_{DD}$    (T=25°C)          | 2.375 V – 3.46 V               |
| $V_{LOGIC}$ (T=25°C)          | 1.8 V $\pm$ 5%                 |


<img src="dinamic_range_gyroscope.png" alt="specifiche giroscopio" width="800"/>

*Figura 4: range dinamico del giroscopio; le misure vengono salvate in due byte (Word Lenght = 16 bits)*

<img src="sensitivity_gyroscope.png" alt="sensibilità giroscopio" width="800"/>

*Figura 5: sensibilità del giroscopio in funzione del fondoscala selezionato.*




### Task 3 – Montaggio del sensore e verifiche di base

**Obiettivo:**
- Comprendere il significato dei pin della breakout del MPU6050.
- Verificare che l’alimentazione disponibile (3.3 V di AD2) sia compatibile con il sensore.
- Collegare correttamente i pin I²C (SCL, SDA) ad AD2 e determinare sperimentalmente l’indirizzo I²C (SAD) tramite uno script di scansione.

**Descrizione:**
- Si usa il pin-out e lo schema di tipica applicazione del datasheet (sezione "Pin Out and Signal Description") per identificare i pin VCC, GND, SCL, SDA (ed eventuale AD0).
- Si collega la breakout del sensore all’alimentazione 3.3 V di AD2 e alle linee digitali (es. DIO0 per SCL e DIO1 per SDA).
- Si scrive uno script Python che:
  - inizializza AD2 e il bus I²C (`i2c = tdwf.I2Cbus(ad2.hdwf)`),
  - chiama `i2c.scan()` e stampa la lista di indirizzi trovati.
- Si confronta l’indirizzo ottenuto con quello atteso dal datasheet (0x68 o 0x69 a seconda di AD0) e lo si annota come valore di SAD per l’esperienza.










### Task 4 – Comandi, registri e conversione in unità fisiche

**Obiettivo:**
- Individuare i **registri principali** del MPU6050:
  - registri di configurazione (fondoscala, filtri, frequenze di campionamento),
  - registri di gestione (es. PWR_MGMT_1),
  - registri dati per accelerometro, giroscopio e temperatura.
- Annotare le **formule** per convertire i dati grezzi (LSB) in unità fisiche (g, °/s, °C).

**Descrizione:**
- Si usa il datasheet e (eventualmente) il documento di Register Map (citato nel datasheet) per:
  - elencare i registri di configurazione più utili (es. `PWR_MGMT_1`, `GYRO_CONFIG`, `ACCEL_CONFIG`, `CONFIG`, `SMPLRT_DIV`),
  - elencare gli indirizzi dei registri di dati (coppie high/low per ciascun asse e per la temperatura).
- Si annotano:
  - i bit che controllano il fondoscala (AFS_SEL per l’accelerometro, FS_SEL per il giroscopio),
  - i valori di fondoscala e scale factor corrispondenti.
- Si scrivono le formule generali:
  - $a[g] = \text{raw} / S_\text{acc}$,
  - $\omega[^\circ/s] = \text{raw} / S_\text{gyro}$,
  - $T[^\circ\mathrm{C}]$ in funzione di `TEMP_raw` e dei parametri di sensibilità/offset.
- Come supporto opzionale, si interroga una AI generativa con un prompt strutturato per farsi aiutare a costruire una tabella dei registri e si confrontano criticamente le risposte con il datasheet.





### Task 5 – Script di lettura periodica (I2C_MPU6050_template)

**Obiettivo:**
- Modificare lo script `I2C_MPU6050_template` fornito dal corso per:
  - inizializzare correttamente il MPU6050 (uscita dallo stand-by, fondoscala desiderati),
  - leggere in modo periodico i registri di misura,
  - convertire i dati grezzi in unità fisiche,
  - stampare o salvare le misure per l’analisi successiva.

**Descrizione:**
- Si parte dal notebook/template che già comunica con il sensore e legge almeno la temperatura.
- Si aggiungono:
  - la scrittura nel registro `PWR_MGMT_1` per svegliare il sensore,
  - la configurazione dei fondoscala di accelerometro e giroscopio (scrittura nei registri `ACCEL_CONFIG` e `GYRO_CONFIG`),
  - funzioni di utilità per leggere parole a 16 bit (high + low) e convertirle in interi con segno (two’s complement),
  - la conversione in unità fisiche usando i fattori di scala del Task 2.
- Si implementa un ciclo principale che:
  - legge le misure sugli assi $(a_x, a_y, a_z)$, $(\omega_x, \omega_y, \omega_z)$ e la temperatura,
  - stampa i valori in formato leggibile e/o li scrive su file (es. CSV) per l’analisi statistica nel Task 6.





### Task 6 – Studio statistico e esperimenti di fisica

**Obiettivo:**
- Studiare la **precisione** e la **distribuzione** delle misure del sensore (accelerazioni, velocità angolari, temperatura).
- Valutare la presenza di **offset** (zero-g, zero-rate), **drift** nel tempo e la dipendenza dalle impostazioni di acquisizione.
- Utilizzare il sensore per almeno un **esperimento di fisica**, ad esempio:
  - misura di $g$ a sensore fermo,
  - misura dell’offset del giroscopio (nulling),
  - stima del tempo di smorzamento di un pendolo/altalena,
  - contapassi rudimentale.

**Descrizione:**
- Usando lo script del Task 5 si acquisiscono serie temporali di:
  - accelerazione (es. lungo l’asse $z$ a sensore fermo per misurare $g$),
  - velocità angolare (es. asse $z$ a sensore fermo per misurare l’offset del giroscopio),
  - temperatura nel tempo dopo l’accensione per stimare il tempo di stabilizzazione.
- Si costruiscono:
  - istogrammi delle misure (per stimare media, deviazione standard, forma della distribuzione),
  - grafici nel tempo (per osservare drift e transitori).
- Si confrontano:
  - la media di $g$ con il valore atteso (1 g) e con la **Initial Calibration Tolerance**,
  - l’offset e il rumore del giroscopio con le specifiche del datasheet,
  - il tempo di stabilizzazione della temperatura con un modello esponenziale.
- Per l’esperimento di fisica scelto (pendolo invertito):
  - si progetta una configurazione meccanica (es. sensore fissato al corpo),
  - si acquisisce una serie temporale,
  - si estraggono parametri fisici (es. modi normali) usando opportune elaborazioni FFt, ecc.).